In [1]:
!pip install nlp
!pip install shap
!pip install transformers
!pip install transformers-interpret

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install lime

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [4]:
import torch
import pandas as pd
import scipy as sp
import numpy as np
import torch.nn as nn
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, Dataset

In [5]:
from lime.lime_text import LimeTextExplainer

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
num_classes = 2
num_epochs = 101
batch_size = 32
hidden_dim = 256
max_length = 256

In [8]:
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, hidden_dim),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(hidden_dim, num_classes)
)
model.load_state_dict(torch.load("./models/biobert_fine_tuned_epoch_16.pt")['model_state_dict'])
model.to(device)
print()

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [63]:
import torch.nn as nn
# def f(x):
#     tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=128, truncation=True) for v in x]).cuda()
#     attention_mask = (tv!=0).type(torch.int64).cuda()
#     outputs = model(tv,attention_mask=attention_mask)[0].detach().cpu().numpy()
#     scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
#     val = sp.special.logit(scores[:,1])
#     return val

# def f(text):
#     print("Calling function")
#     print(text)
#     with torch.no_grad():
#         #tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=max_length, truncation=True) for v in texts]).cuda()
#         tv = torch.tensor([tokenizer.encode(text, padding='max_length', max_length=max_length, truncation=True)]).cuda()
#         a_mask = (tv!=0).type(torch.int64).cuda()
#         outputs = model(tv,attention_mask=a_mask)[0].cpu()
#         outputs = nn.functional.softmax(outputs, dim=1).numpy()
#         return outputs.tolist()

def f(texts):
    print("Calling function")
    print(texts)
    outputs = []
    for text in texts:
        with torch.no_grad():
            #tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=max_length, truncation=True) for v in texts]).cuda()
            tv = torch.tensor([tokenizer.encode(text, padding='max_length', max_length=max_length, truncation=True)]).cuda()
            a_mask = (tv!=0).type(torch.int64).cuda()
            output = model(tv,attention_mask=a_mask)[0].cpu()
            output = nn.functional.softmax(output, dim=1).numpy()
            outputs.append(output)
    print(np.array(outputs[0]))
    return np.array(outputs[0])

In [64]:
example = example = ('giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium', 1)
text, label = example
f([text])

Calling function
['giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium']
[[0.1233345  0.87666553]]


array([[0.1233345 , 0.87666553]], dtype=float32)

In [65]:
# build an explainer using a token masker
example = example = ("giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium", 1)
text, label = example
class_names = ['non_cardiovascular', 'cardiovascular']
explainer = LimeTextExplainer(class_names=class_names)
explainer.explain_instance(text, f, num_features=6)

Calling function
['giant left atriuma case report seventysevenyear old woman with mitral stenosis presented with cardiomegaly evident her chest roentgenogram cardiac enlargement due giant left atrium that distorted cardiac structures echocardiogram and firstpass nuclear angiogram able delineate huge left atrium', '               her chest roentgenogram                    ', 'giant left atriuma case    woman with    with    chest roentgenogram  enlargement  giant left  that    echocardiogram  firstpass nuclear   delineate  left ', '   case  seventysevenyear old   mitral               that     and   angiogram  delineate   ', '           presented   evident      due    that    echocardiogram and      huge  ', 'giant                     giant         firstpass       ', ' left  case report seventysevenyear old  with mitral   with cardiomegaly evident   roentgenogram cardiac    left  that  cardiac structures echocardiogram and   angiogram  delineate  left ', 'giant  atriuma case report  old 

[[0.15582225 0.8441778 ]]


ValueError: Found input variables with inconsistent numbers of samples: [5000, 1]